<a href="https://colab.research.google.com/github/Pandagan-85/ZTM-Machine-learning/blob/main/end_to_end_dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐶 End-to-end Multi-Class Dog Breed Classification
This notebook builds an end-to-end mult-class image classifier using TensorFlow 2.0 and TensorFlow Hub.

## 1. Problem
Identifying the breed of a dog given an image of a dog.

When I'm sitting at the cafe and i take a photo of a dog, I want to know what breed of dog it is.

## 2. Data

The Data we're using is from Kaggle's dog breed identification competition. [qui](https://www.kaggle.com/c/dog-breed-identification/data)

## 3. Evaluation
The evaluation is a file with prediction probabilities for each dog breed for each test image. [qui](www.kaggle.com/competitions/dog-breed-identification/overview/evaluation)
```
id,affenpinscher,afghan_hound,..,yorkshire_terrier
000621fb3cbb32d8935728e48679680e,0.0083,0.0,...,0.0083
etc.
```

## 4. Features
Some information about the data:
* We're dealing with images(unstructured data) so it's problaby best we use deep learning/transfer learning.
* There are 120 breeds of dogs (this means there are 120 different classes).
* There are around 10,000+ images in the training set (These images have labels)
* There are around 10,000+ images in the test set  (These images have no labels, because we'll want to predict them.)

In [3]:
# Unzip the uploaded data to Google Drive
#!unzip "drive/MyDrive/Dog Vision/dog-breed-identification.zip" -d "drive/MyDrive/Dog Vision/"

## Get out workspace ready
* Import TensorFlow 2.x ✅
* Import TensorFlow Hub
* Make sure we're using a GPU

In [2]:
# Import TensorFlow into Colab
import tensorflow as tf
print("TF version:", tf.__version__)

TF version: 2.18.0
